## PYNQ test hls4ml NN IP
This notebook is a performance & resource comparison between the system passing encoded integer values vs the system passing floating point values, delegating the decoding/encoding on the FPGA. 

### Drivers definition

In [2]:
from pynq import DefaultHierarchy, DefaultIP, allocate
from datetime import datetime
import pynq.lib.dma
import numpy as np


class AxiLiteNN(DefaultIP):
    """
    Basic driver to read/write 'vector_rows' register of the accelerator using AxiLite protocol.
    The address for reading/writing is taken from the .h driver file produced by vivado_hls.
    """
    def __init__(self, description):
        super().__init__(description=description)
    
    # can it be written by hls4ml? Should improve flexibility
    bindto = ['CERN:hls4ml:myproject_axi:1.0']

    @property
    def vector_rows(self):
        return self.read(0x10)

    @vector_rows.setter
    def vector_rows(self, value):
        self.write(0x10, value)

class NN(DefaultHierarchy):
    """
    Hierarchy driver related to the hierarchy composed by the AXI DMA module + the HLS accelerator.
    It uses the AxiLiteNN class to write the number of rows of the input matrix to the accelerator.
    """
    def __init__(self, description):
        super().__init__(description=description)
    
    def _print_dt(self, timea, timeb, N):
        dt = (timeb - timea) 
        dts = dt.seconds + dt.microseconds * 10**-6
        rate = N / dts
        print("Classified {} samples in {} seconds ({} inferences / s)".format(N, dts, rate))
        return dts, rate
    
    def predict(self, X, y_shape, dtype=np.float32, profile=False, encode=None, decode=None):
        """
        Obtain the predictions of the NN implemented in the FPGA.
        Parameters:
        - X : the input vector. Should be numpy ndarray.
        - y_shape : the shape of the output vector. Needed to the accelerator to set the TLAST bit properly and
                    for sizing the output vector shape.
        - dtype : the data type of the elements of the input/output vectors. 
                  Note: it should be set depending on the interface of the accelerator; if it uses 'float' 
                  types for the 'data' AXI-Stream field, 'np.float32' dtype is the correct one to use. 
                  Instead if it uses 'ap_fixed<A,B>', 'np.intA' is the correct one to use (note that A cannot
                  any integer value, but it can assume {..., 8, 16, 32, ...} values. Check `numpy` 
                  doc for more info).
                  In this case the encoding/decoding has to be computed by the PS. For example for 
                  'ap_fixed<16,6>' type the following 2 functions are the correct one to use for encode/decode 
                  'float' -> 'ap_fixed<16,6>':
                  
                  ```
                    def encode(xi):
                        return np.int16(round(xi * 2**10)) # note 2**10 = 2**(A-B)

                    def decode(yi):
                        return yi * 2**-10

                    encode_v = np.vectorize(encode) # to apply them element-wise
                    decode_v = np.vectorize(decode)
                  ```
        - profile : boolean. Set it to `True` to print the performance of the algorithm in term of `inference/s`.
        - encode/decode: function pointers. See `dtype` section for more information.
        - return: an output array based on `np.ndarray` with a shape equal to `y_shape` and a `dtype` equal to
                  the namesake parameter.
        
        """
        if profile:
            timea = datetime.now()
        if encode is not None:
            X = encode(X)
        self.myproject_axi_0.vector_rows = X.shape[0]
        with allocate(shape=X.shape, dtype=dtype) as input_buffer, \
             allocate(shape=y_shape, dtype=dtype) as output_buffer:
            input_buffer[:] = X 
            self.axi_dma_0.sendchannel.transfer(input_buffer)
            self.axi_dma_0.recvchannel.transfer(output_buffer)
            self.axi_dma_0.sendchannel.wait()
            self.axi_dma_0.recvchannel.wait()
            result = output_buffer.copy()
        if decode is not None:
            result = decode(result)
        if profile:
            timeb = datetime.now()
            dts, rate = self._print_dt(timea, timeb, len(X))
            return result, dts, rate
        return result
    
    @staticmethod
    def checkhierarchy(description):
        # can they be written by hls4ml? Should improve flexibility
        if 'axi_dma_0' in description['ip'] \
           and 'myproject_axi_0' in description['ip']:
            return True
        return False

## Load bitfile (overlay)

In [3]:
from pynq import Overlay

overlay = Overlay("./nn_float.bit")

## Inspect the structure of the overlay
To pretty-print and reduce elements of the json output, copy-paste the code [here](https://jsonformatter.curiousconcept.com/)

In [4]:
if overlay.is_loaded():
    print(overlay.ip_dict)

{'hier_0/axi_dma_0': {'fullpath': 'hier_0/axi_dma_0', 'type': 'xilinx.com:ip:axi_dma:7.1', 'state': None, 'addr_range': 65536, 'phys_addr': 1077936128, 'mem_id': 'S_AXI_LITE', 'gpio': {}, 'interrupts': {}, 'parameters': {'C_S_AXI_LITE_ADDR_WIDTH': '10', 'C_S_AXI_LITE_DATA_WIDTH': '32', 'C_DLYTMR_RESOLUTION': '125', 'C_PRMRY_IS_ACLK_ASYNC': '0', 'C_ENABLE_MULTI_CHANNEL': '0', 'C_NUM_MM2S_CHANNELS': '1', 'C_NUM_S2MM_CHANNELS': '1', 'C_INCLUDE_SG': '0', 'C_SG_INCLUDE_STSCNTRL_STRM': '0', 'C_SG_USE_STSAPP_LENGTH': '0', 'C_SG_LENGTH_WIDTH': '26', 'C_M_AXI_SG_ADDR_WIDTH': '32', 'C_M_AXI_SG_DATA_WIDTH': '32', 'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32', 'C_S_AXIS_S2MM_STS_TDATA_WIDTH': '32', 'C_MICRO_DMA': '0', 'C_INCLUDE_MM2S': '1', 'C_INCLUDE_MM2S_SF': '1', 'C_MM2S_BURST_SIZE': '128', 'C_M_AXI_MM2S_ADDR_WIDTH': '32', 'C_M_AXI_MM2S_DATA_WIDTH': '32', 'C_M_AXIS_MM2S_TDATA_WIDTH': '32', 'C_INCLUDE_MM2S_DRE': '0', 'C_INCLUDE_S2MM': '1', 'C_INCLUDE_S2MM_SF': '1', 'C_S2MM_BURST_SIZE': '128', 'C_M_AXI

## Create objects from elements in the overlay dictionary
Instantiate NN class. Check if the driver is working by looking at the `help` output; the first lines should be like `class NN(pynq.overlay.DefaultHierarchy)`

In [5]:
import pynq.lib.dma

nn = overlay.hier_0
help(nn)
help(nn.myproject_axi_0)

Help on NN in module __main__ object:

class NN(pynq.overlay.DefaultHierarchy)
 |  Hierarchy driver related to the hierarchy composed by the AXI DMA module + the HLS accelerator.
 |  It uses the AxiLiteNN class to write the number of rows of the input matrix to the accelerator.
 |  
 |  Method resolution order:
 |      NN
 |      pynq.overlay.DefaultHierarchy
 |      pynq.overlay._IPMap
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, description)
 |      Create a new _IPMap based on a hierarchical description.
 |  
 |  predict(self, X, y_shape, dtype=<class 'numpy.float32'>, profile=False, encode=None, decode=None)
 |      Obtain the predictions of the NN implemented in the FPGA.
 |      Parameters:
 |      - X : the input vector. Should be numpy ndarray.
 |      - y_shape : the shape of the output vector. Needed to the accelerator to set the TLAST bit properly and
 |                  for sizing the output vector shape.
 |      - dtype : the data type of 

## Prepare input/output data
- `y_hls.npy` contains the output generated by the C simulation of the NN (that in principle should be equal to the predictions generated by the IP running on the FPGA).
- `X_test.npy` file represents NN the input data.

In [6]:
y_hls = np.load('./y_hls.npy').astype(np.float32)
X = np.load('./X_test.npy').astype(np.float32)

## Let's perform a prediction on the PL!

In [7]:
y_float, dts_float, rate_float = nn.predict(X, y_hls.shape, profile=True)
print(y_float)

Classified 166000 samples in 0.8894449999999999 seconds (186633.23758073855 inferences / s)
[[ 0.45214844  0.31054688  0.03027344  0.14648438  0.15625   ]
 [ 0.07128906  0.68066406  0.01367188  0.15136719  0.09765625]
 [ 0.11425781  0.05664062  0.84570312  0.          0.00585938]
 ..., 
 [ 0.01757812  0.06640625  0.07128906  0.82128906  0.0625    ]
 [ 0.03710938  0.109375    0.86425781  0.          0.00585938]
 [ 0.0625      0.05566406  0.1171875   0.67871094  0.0859375 ]]


## Output check
Check if the data generated by the NN IP are the same as the one obtained via C simulation.

In [8]:
areEqual = np.array_equal(y_hls, y_float)
print("Array are equal: {}".format(areEqual))
if not areEqual:
    dy = np.abs(y_hls - y_float)
    print("Difference values: {}, indexes: {}".format(dy[np.where(dy>0)], np.where(dy>0)))
    print("Max absolute difference: {}".format(dy.max()))
    print("Max relative difference: {}".format((dy[np.where(y_hls > 0)] / np.abs(y_hls[np.where(y_hls > 0)])).max()))

Array are equal: False
Difference values: [ 0.00097656  0.00292969  0.00390625], indexes: (array([61143, 61143, 61143], dtype=int32), array([0, 1, 2], dtype=int32))
Max absolute difference: 0.00390625
Max relative difference: 0.0714285746216774


## Same code but different interface - `ap_fixed<16,6>`
The cells you can run from this point on are the same as the one before. The difference is that the accelerator this time expects `ap_fixed<16,6>` data instead of `float`. This time an encoding/decoding function is required. Check the `predict` cell to see how the driver handle also this case.

In [9]:
overlay = Overlay("./nn_fixed.bit")
nn = overlay.hier_0
help(nn)

Help on NN in module __main__ object:

class NN(pynq.overlay.DefaultHierarchy)
 |  Hierarchy driver related to the hierarchy composed by the AXI DMA module + the HLS accelerator.
 |  It uses the AxiLiteNN class to write the number of rows of the input matrix to the accelerator.
 |  
 |  Method resolution order:
 |      NN
 |      pynq.overlay.DefaultHierarchy
 |      pynq.overlay._IPMap
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, description)
 |      Create a new _IPMap based on a hierarchical description.
 |  
 |  predict(self, X, y_shape, dtype=<class 'numpy.float32'>, profile=False, encode=None, decode=None)
 |      Obtain the predictions of the NN implemented in the FPGA.
 |      Parameters:
 |      - X : the input vector. Should be numpy ndarray.
 |      - y_shape : the shape of the output vector. Needed to the accelerator to set the TLAST bit properly and
 |                  for sizing the output vector shape.
 |      - dtype : the data type of 

In [10]:
def encode(xi):
    return np.int16(round(xi * 2**10))

def decode(yi):
    return yi * 2**-10

encode_v = np.vectorize(encode)
decode_v = np.vectorize(decode)

In [11]:
y_fixed, dts_fixed, rate_fixed = nn.predict(X, y_hls.shape, dtype=np.int16, profile=True, encode=encode_v, decode=decode_v)
print(y_fixed)

Classified 166000 samples in 51.427612 seconds (3227.8379948888155 inferences / s)
[[ 0.45214844  0.31054688  0.03027344  0.14648438  0.15625   ]
 [ 0.07128906  0.68066406  0.01367188  0.15136719  0.09765625]
 [ 0.11425781  0.05664062  0.84570312  0.          0.00585938]
 ..., 
 [ 0.01757812  0.06640625  0.07128906  0.82128906  0.0625    ]
 [ 0.03710938  0.109375    0.86425781  0.          0.00585938]
 [ 0.0625      0.05566406  0.1171875   0.67871094  0.0859375 ]]


In [12]:
areEqual = np.array_equal(y_hls, y_fixed)
print("Array are equal: {}".format(areEqual))
if not areEqual:
    dy = np.abs(y_hls - y_fixed)
    print("Difference values: {}, indexes: {}".format(dy[np.where(dy>0)], np.where(dy>0)))
    print("Max absolute difference: {}".format(dy.max()))
    print("Max relative difference: {}".format((dy[np.where(y_hls > 0)] / np.abs(y_hls[np.where(y_hls > 0)])).max()))

Array are equal: False
Difference values: [ 0.00097656  0.01074219  0.00390625 ...,  0.00683594  0.00878906
  0.00585938], indexes: (array([     9,      9,      9, ..., 165983, 165983, 165996], dtype=int32), array([0, 3, 4, ..., 1, 4, 1], dtype=int32))
Max absolute difference: 0.166015625
Max relative difference: 1.0


## Same code but different interface - `hls::stream`

In [13]:
overlay = Overlay("./nn_stream.bit")
nn = overlay.hier_0
help(nn)

Help on NN in module __main__ object:

class NN(pynq.overlay.DefaultHierarchy)
 |  Hierarchy driver related to the hierarchy composed by the AXI DMA module + the HLS accelerator.
 |  It uses the AxiLiteNN class to write the number of rows of the input matrix to the accelerator.
 |  
 |  Method resolution order:
 |      NN
 |      pynq.overlay.DefaultHierarchy
 |      pynq.overlay._IPMap
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, description)
 |      Create a new _IPMap based on a hierarchical description.
 |  
 |  predict(self, X, y_shape, dtype=<class 'numpy.float32'>, profile=False, encode=None, decode=None)
 |      Obtain the predictions of the NN implemented in the FPGA.
 |      Parameters:
 |      - X : the input vector. Should be numpy ndarray.
 |      - y_shape : the shape of the output vector. Needed to the accelerator to set the TLAST bit properly and
 |                  for sizing the output vector shape.
 |      - dtype : the data type of 

In [14]:
y_stream, dts_stream, rate_stream = nn.predict(X, y_hls.shape, profile=True)
print(y_float)

Classified 166000 samples in 0.926368 seconds (179194.44540398632 inferences / s)
[[ 0.45214844  0.31054688  0.03027344  0.14648438  0.15625   ]
 [ 0.07128906  0.68066406  0.01367188  0.15136719  0.09765625]
 [ 0.11425781  0.05664062  0.84570312  0.          0.00585938]
 ..., 
 [ 0.01757812  0.06640625  0.07128906  0.82128906  0.0625    ]
 [ 0.03710938  0.109375    0.86425781  0.          0.00585938]
 [ 0.0625      0.05566406  0.1171875   0.67871094  0.0859375 ]]


In [15]:
areEqual = np.array_equal(y_hls, y_stream)
print("Array are equal: {}".format(areEqual))
if not areEqual:
    dy = np.abs(y_hls - y_stream)
    print("Difference values: {}, indexes: {}".format(dy[np.where(dy>0)], np.where(dy>0)))
    print("Max absolute difference: {}".format(dy.max()))
    print("Max relative difference: {}".format((dy[np.where(y_hls > 0)] / np.abs(y_hls[np.where(y_hls > 0)])).max()))

Array are equal: False
Difference values: [ 0.00097656  0.00292969  0.00390625], indexes: (array([61143, 61143, 61143], dtype=int32), array([0, 1, 2], dtype=int32))
Max absolute difference: 0.00390625
Max relative difference: 0.0714285746216774


In [16]:
print("Rate comparison")
print("Fixed point rate:       {} inferences/s".format(rate_fixed))
print("Floating point rate:    {} inferences/s".format(rate_float))
print("Stream rate:            {} inferences/s".format(rate_stream))
print("Floating point speedup: {}x".format(round(dts_fixed/dts_float)))
print("Stream speedup:         {}x".format(round(dts_fixed/dts_stream)))

Rate comparison
Fixed point rate:       3227.8379948888155 inferences/s
Floating point rate:    186633.23758073855 inferences/s
Stream rate:            179194.44540398632 inferences/s
Floating point speedup: 58x
Stream speedup:         56x


## Results
Floating-point version is approx. 60x faster than fixed point one. Delegating the encode/decode part to the PL seems effective for the throughput. Let's make a resource utilization comparison (taken from Vivado reports).

| Interface | LUTs | SRLs | FFs | RAMB36 | RAMB18 | DSP48 |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| Floating-point 32 | 18321(34,442970%) | 266(1,533736%) | 19049(17,908195%) | 2(1,433571%) | 6(2,147857%) | 21(9,550455%) |
| Fixed-point 16 |  17654(33,189211%) | 257(1,482011%) | 18333(17,235263%) | 2(1,433571%) | 6(2,147857%) | 21(9,550455%) |
| Stream FP32 | 17192(32,320789%) | 387(2,229138%) | 19167(18,019098%) | 2(1,433571%) | 6(2,147857%) | 21(9,550455%) |